In [1]:
import os 
import sys

In [2]:
%pwd

'/config/workspace/notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/config/workspace'

In [5]:
""" os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/gbiamgaurav/South_German_Credit_Risk.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="gbiamgaurav" 
os.environ["MLFLOW_TRACKING_PASSWORD"]="352861024ebede76aa75cf4ea021ee1f74b0e39f" """

' os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/gbiamgaurav/South_German_Credit_Risk.mlflow"\nos.environ["MLFLOW_TRACKING_USERNAME"]="gbiamgaurav" \nos.environ["MLFLOW_TRACKING_PASSWORD"]="352861024ebede76aa75cf4ea021ee1f74b0e39f" '

### Create Entity

In [6]:
from dataclasses import dataclass
from pathlib import Path 


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path 
    test_data_path: Path 
    model_path: Path 
    all_params: dict 
    metric_file_name: Path 
    target_column: str 

### Create ConfigurationManager

In [7]:
from South_German_Bank.constants import * 
from South_German_Bank.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBClassifier
        schema = self.schema.TARGET_COLUMN 

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            
        )

        return model_evaluation_config

### Create Components

In [9]:
import os 
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from urllib.parse import urlparse
import json
import numpy as np 
import joblib
from South_German_Bank.constants import * 
from South_German_Bank.utils.common import read_yaml, create_directories, save_json

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        acc = accuracy_score(actual, pred)
        f1 = f1_score(actual, pred)
        p_score = precision_score(actual, pred)
        r_score = recall_score(actual, pred)
        ras = roc_auc_score(actual, pred)
        return acc, f1, p_score, r_score, ras

    def log_metrics_locally(self, acc, f1, p_score, r_score, ras):
        scores = {
            "accuracy_score": acc,
            "f1_score": f1,
            "precision_score": p_score,
            "recall_score": r_score,
            "roc_auc_score": ras
        }
        with open(self.config.metric_file_name, "w") as f:
            json.dump(scores, f)

    def evaluate_model(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop(self.config.target_column, axis=1)
        y_test = test_data[[self.config.target_column]]

        predicted_qualities = model.predict(X_test)

        acc, f1, p_score, r_score, ras = self.eval_metrics(y_test, predicted_qualities)

        self.log_metrics_locally(acc, f1, p_score, r_score, ras)

### Create Pipeline

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate_model()
except Exception as e:
    raise e

[{asctime}s: INFO: common: yaml file: config/config.yaml loaded successfully]
[{asctime}s: INFO: common: yaml file: params.yaml loaded successfully]
[{asctime}s: INFO: common: yaml file: schema.yaml loaded successfully]
[{asctime}s: INFO: common: created directory at: artifacts]
[{asctime}s: INFO: common: created directory at: artifacts/model_evaluation]
